In [1]:
%matplotlib ipympl

# calculation
import numpy as np

# plotting
import matplotlib
import matplotlib.pyplot as plt

# arguments
import argparse

# mcphonon classes
from classes.Geometry import Geometry
from classes.Phonon import Phonon
from classes.Population import Population
from classes.Visualisation import Visualisation

class Args:
    def __init__(self, dict):
        self.__dict__.update(dict)
        
        array_keys = ['dimensions', 'scale' , 'rotation'    , 'particles'     , 'timestep',
                    'iterations', 'slices', 'temperatures', 'threshold_temp']
        type = ['float'     , 'float' , 'float'       , 'int'           , 'float'   ,
                'int'       , 'int'   , 'float'       , 'float'         ]
        path_keys = ['poscar_file', 'hdf_file', 'results_location', 'results_folder']
        
        all_keys = [i for i in self.__dict__.keys()]

        for i in range(len(all_keys)):
            key = all_keys[i]
            arg = self.__dict__[key]
            if key in array_keys:
                index = array_keys.index(key)
                self.__dict__[key] = np.fromstring(arg, sep=' ').astype(type[index])
            elif key in path_keys:
                pass
            else:
                self.__dict__[key] = [arg]

In [2]:
plt.close('all')

# Set folder to be analysed

folders = ['D:/LEMTA/Code/Results/CSI_1/']

# Read arguments

T_profiles_mean = []
T_profiles_error = []
T_difference_mean = []
space_axis = []

for folder in folders:
    print('Processing folder:', folder)

    arg_file = open(folder+'arguments.txt', 'r')

    lines = [i.strip()          for i in arg_file.readlines()]
    lines = [i.replace('[', '') for i in lines]
    lines = [i.replace(']', '') for i in lines]
    lines = [i.replace("'", '') for i in lines]
    lines = [i.replace(",", '') for i in lines]
    lines = [i.split(' = ')     for i in lines]

    arg_file.close()

    arg_dict = {i[0]:i[1] for i in lines if len(i)>1}

    args = Args(arg_dict)

    # Loading phonon data

    phonon = Phonon(args)
    phonon.load_properties()

    geo = Geometry(args)

    view = Visualisation(args, geo, phonon)
    view.read_convergence()
    view.read_particles()
    # view.convergence_temperature()
    # view.convergence_heat_flux()
    # view.convergence_particles()
    # view.convergence_energy_balance()
    view.energy_dispersion()

    n_dt_to_conv = np.floor( np.log10( args.iterations[0] ) ) - 2    # number of timesteps to save convergence data
    n_dt_to_conv = int(10**n_dt_to_conv)
    n_dt_to_conv = max([10, n_dt_to_conv])

    n = int(200/n_dt_to_conv)

    #normalised
    # T_array = (view.T[-n:, :].mean(axis = 0)-args.temperatures.min())/args.temperatures.ptp()
    
    # not normalised
    T_array = view.T[-n:, :].mean(axis = 0)
    T_std   = view.T[-n:, :].std(axis = 0)

    T_profiles_mean.append(T_array)
    T_profiles_error.append(T_std)
    
    T_sup_array = np.ones(T_array.shape[0]+2)
    T_sup_array[1:-1]=T_array
    T_sup_array[[0, -1]] = args.temperatures
    T_moving_mean = np.convolve(T_sup_array, np.array([1, 0, 1])/2, mode='valid')
    
    T_difference_mean.append(T_moving_mean)

    x_axis = np.arange(geo.n_of_slices)*geo.slice_length+geo.slice_length/2
    x_axis = x_axis/(geo.n_of_slices*geo.slice_length)

    space_axis.append(x_axis)
    
print('Done!')

Processing folder: D:/LEMTA/Code/Results/CSI_1/
Reading hdf file...
Expanding frequency to FBZ...
Expanding group velocity to FBZ...
Expanding gamma to FBZ...
To describe phonons:
 nq= 29791
 nb= 6
 number of modes= 178746
Interpolating lifetime...
Generating T = f(E)...
Material initialisation done!
Loading geometry...
Transforming geometry...
Slicing domain...
Geometry processing done!
Initialising visualisation class...
Start reading particle data
Finished reading particle data


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Done!


In [ ]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)

sl = 4
T = view.T[-1, sl]

BE_occupation = phonon.calculate_occupation(T, phonon.omega, threshold = True)

indexes = view.slice_id == sl

NE_occupation = view.occupation[indexes] #+ phonon.calculate_occupation(phonon.T_threshold, phonon.omega, threshold= False)[view.q_point[indexes], view.branch[indexes]]

bins = 1000
# _ = ax.hist(np.log10(BE_occupation[BE_occupation>0]), bins = bins, density = True, color = 'b', histtype = 'step')
# _ = ax.hist(np.log10(NE_occupation[NE_occupation>0]), bins = bins, density = True, color = 'r')

_ = ax.scatter(phonon.omega[view.q_point[indexes], view.branch[indexes]], NE_occupation, color = 'r', s = 1)
_ = ax.scatter(phonon.omega.reshape(-1), BE_occupation.reshape(-1), color = 'b', s = 1)
ax.set_yscale('log')

# ax.set_xlim(-3, -1)




In [ ]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(111)



ax.plot([0, 1],#[(-geo.slice_length/2)/(geo.n_of_slices*geo.slice_length), (geo.n_of_slices+1/2)*geo.slice_length/(geo.n_of_slices*geo.slice_length)],
        args.temperatures,
        color = 'gray', linestyle = '--', alpha = 0.5)

ax1 = ax.twinx()

for i in range(len(folders)):
        # ax.plot(space_axis[i], T_profiles_mean[i], '+-')
        ax.errorbar(space_axis[i], T_profiles_mean[i], yerr = 3*T_profiles_error[i], capsize = 2, linewidth = 1)
        ax1.semilogy(space_axis[i], np.absolute(T_profiles_mean[i]-T_difference_mean[i]), 'o', color = 'k')

# legends = ['Linear']
legends = [i.replace('D:/LEMTA/Code/Results/', '').replace('_0/', '').replace('_1/', '') for i in folders]

ax.legend(legends)

ax.set_title('Mean final heat flux (last {}ps)'.format(200))
ax.set_ylabel('Heat Flux [W/m²]')
ax.set_xlabel('Normalised position [-]')

In [ ]:
# Plotting v_g x omega

slice_axis = args.slices[1]
T = 303

flat_omega = phonon.omega[phonon.unique_modes[:, 0], phonon.unique_modes[:, 1]]

flat_group_vel = phonon.group_vel[phonon.unique_modes[:, 0], phonon.unique_modes[:, 1], slice_axis]
flat_group_vel = np.around(flat_group_vel, 3)

flat_energy = phonon.calculate_energy(T, flat_omega)

flat_occupation = phonon.calculate_occupation(T, flat_omega)

plt.close('all')
fig = plt.figure(figsize = (10, 5), dpi = 120)
ax = fig.add_subplot(121)
ax.scatter(flat_group_vel, flat_omega, s=1, c = phonon.unique_modes[:, 1])
ax1 = ax.twinx()
_ = ax1.hist(flat_group_vel, bins = 100, alpha = 0.5, density = True)

ax.set_xlabel('Group velocity [ang/ps]')
ax.set_ylabel('Angular frequency [rad THz]')
ax1.set_ylabel('Frequency of appearance')

dt = 0.5
dx_domain = geo.bounds[:, slice_axis].ptp()
enter_prob = np.where(np.absolute(flat_group_vel) > 0, (dt * np.absolute(flat_group_vel))/geo.slice_length, 0)
enter_prob = np.where(enter_prob > 1, 1, enter_prob)
n_it_to_enter = np.where(enter_prob == 0, 0, np.ceil(args.slices[0]/enter_prob))

n_it_to_leave = np.ceil(dx_domain/np.absolute(flat_group_vel*dt))

index_order = np.argsort(n_it_to_enter)
sorted_energy = flat_energy[index_order]
cumulated_energy = np.cumsum(sorted_energy)/sorted_energy.sum()

print( n_it_to_enter[index_order][(cumulated_energy>=0.9999)][0])

# ax.hist(flat_energy[indexes], bins = 100)

plt.tight_layout()


In [ ]:
# lifetime analysis

dt = 0.01

slice_axis = args.slices[1]
T = 303

flat_omega = phonon.omega[phonon.unique_modes[:, 0], phonon.unique_modes[:, 1]]

flat_group_vel = phonon.group_vel[phonon.unique_modes[:, 0], phonon.unique_modes[:, 1], slice_axis]
flat_group_vel = np.around(flat_group_vel, 3)

T_array = T*np.ones(phonon.number_of_modes).reshape(-1, 1)
Tqj = np.hstack((T_array, phonon.unique_modes))
lifetime = phonon.lifetime_function(Tqj)

# plotting first data

plt.close('all')
fig = plt.figure(figsize = (10, 5), dpi = 120)
ax = fig.add_subplot(121)

ax.fill_between([0, geo.slice_length], 0, geo.slice_length, alpha = 0.2)
ax.scatter(np.absolute(flat_group_vel)*dt, np.absolute(flat_group_vel)*lifetime, marker='.', c=flat_omega, s=1)
ax.plot([1e-3, 1e6], [1e-3, 1e6], color = 'gray', alpha = 0.5, linestyle = '--')
ax.plot([1e-3, 1e6], [0.5e-3, 0.5e6], color = 'gray', alpha = 0.5, linestyle = '--')
# ax.plot([0               , 1e2             ], [geo.slice_length, geo.slice_length], color = 'red' , alpha = 0.5, linestyle = '--')
# ax.plot([geo.slice_length, geo.slice_length], [0               , 1e2             ], color = 'red' , alpha = 0.5, linestyle = '--')


ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel(r'$v_{{g}} \cdot dt ~[\AA]$')
ax.set_ylabel(r'$v_{{g}} \cdot \tau ~[\AA]$')
ax.set_xlim(1e-3, 1e6)
ax.set_ylim(1e-3, 1e6)
ax.text(1.2e-3, geo.slice_length*1.01, s=r'Slice length = {:.2f} $\AA$'.format(geo.slice_length), color = 'gray', fontsize = 'small')
ax.text(geo.slice_length*1.01, 1.5e-3, s=r'Slice length = {:.2f} $\AA$'.format(geo.slice_length), color = 'gray', fontsize = 'small', rotation = 270)
ax.text(1e4, 2e4, s=r'$dt/\tau = 1$', color = 'gray', fontsize = 'small', rotation = 45)
ax.text(2e4, 6e3, s=r'$dt/\tau = 2$', color = 'gray', fontsize = 'small', rotation = 45)
plt.grid()

ax1 = fig.add_subplot(122)

n_it_inside_slice = np.where(np.absolute(flat_group_vel)>0, np.floor(geo.slice_length/(np.absolute(flat_group_vel)*dt)), 0)

def scat(dt, tau, times):
    i = 0
    nu = 0
    
    A = dt/tau
    B = 1-A

    while i<times:
        nu = A + B*nu
        i = i+1
    
    return nu

scattered_nu = np.zeros(flat_omega.shape[0])

print('Scattering...')
for i in range(flat_omega.shape[0]):
    scattered_nu[i] = scat(dt, lifetime[i], n_it_inside_slice[i])
print('Done')

ax1.scatter(n_it_inside_slice, scattered_nu, s = 1, c = flat_omega)
# ax1.set_xscale('log')
ax1.set_xscale('log')
ax1.set_xlabel('Number of iterations inside a slice.')
ax1.set_ylabel('Approximation to the equilibrium occupation.')
ax1.set_ylim(-0.1, 1.1)
ax1.set_xlim(1)
ax1.text(1.5, 1, 'dt = {:.2f} ps'.format(dt))


plt.tight_layout()



In [ ]:
# test of parameters

min_tau =lifetime[lifetime>0].min()
max_vg = np.absolute(flat_group_vel).max()
min_vg = np.absolute(flat_group_vel)[np.absolute(flat_group_vel)>0].min()
dx_domain = 0.5e3

max_dt = min_tau
dt = 0.1

max_n_sl = np.floor(dx_domain/(max_vg*dt))
n_sl = 30

min_prob = min_vg*dt/(dx_domain/n_sl)

print('dt:', dt, 'ps, Max N_sl:', max_n_sl)
print('Max dt: {:.3f} ps'.format(min_tau))
print('Min enter probability: {:.4f}%'.format(min_prob*100))

In [ ]:
# Comparing new and old lifetime interpolation functions

from scipy.interpolate import RegularGridInterpolator, interp1d

q_array = np.arange(phonon.number_of_qpoints )
j_array = np.arange(phonon.number_of_branches)

new_lifetime_function = RegularGridInterpolator((phonon.temperature_array, q_array, j_array), phonon.lifetime)

T = np.linspace(1, 300, 100)
q = int(np.random.rand()*phonon.number_of_qpoints)
j = int(np.random.rand()*phonon.number_of_branches)

old_lifetime_function = interp1d(phonon.temperature_array, phonon.lifetime[:, q, j], kind='linear')

Tqj = np.concatenate((T.reshape(-1, 1),
                      q*np.ones(T.shape[0]).reshape(-1, 1),
                      j*np.ones(T.shape[0]).reshape(-1, 1)), axis = 1)

new_tau = new_lifetime_function(Tqj)
old_tau = old_lifetime_function(T)

plt.close('all')
fig = plt.figure(figsize = (5, 5), dpi = 100)
ax = fig.add_subplot(111)
ax.plot(T, old_tau, '--', c = 'k')
ax.scatter(T, new_tau, marker='+', c = 'r')
ax.set_ylabel('Lifetime [ps]')
ax.set_xlabel('Temperature [K]')
_ = ax.legend(['Original (interp1d)', 'New (RegularGridInterpolator)'])
ax.set_title('q = {}, j = {}'.format(q, j))
plt.tight_layout()
